# Elastic Material

A linear elastic material is one for which

- the stress is a linear function of strain,
- deformation is reversed on reversal of external loading, and
- whose original shape is recovered on removal of external loads

## See Also

- [User Defined Materials](UserMaterial.ipynb)

## Contents

1. <a href='#basic'>Fundamental Equations</a>
2. <a href='#implement'>Model Implementation</a>
3. <a href='#verify'>Model Verification</a> 

<a name='basic'></a>
## Fundamental Equations

The mechanical response of a linear elastic material is defined by

$$
\pmb{\sigma} = \mathbb{C}{:}\pmb{\epsilon}
$$

where $\mathbb{C}$ is the elastic stiffness tensor made up of 21 independent components.  The strain $\pmb{\epsilon}$ can be determined from the deformation gradient $\pmb{F}$ as

$$
\pmb{\epsilon} = \frac{1}{2\kappa}\left[\left(\pmb{F}^{\rm T}{\cdot}\pmb{F}\right)^{2\kappa} - \pmb{I}\right]
$$

where $\kappa$ is the generalized [Seth-Hill](https://en.wikipedia.org/wiki/Finite_strain_theory#Seth-Hill_family_of_generalized_strain_tensors) strain parameter.  Defined as such, several well known finite strain measures are emitted:

- $\kappa=1$: Green-Lagrange reference strain
- $\kappa=-1$: Alamansi spatial strain
- $\kappa=0$: Logarithmic, or true, strain

### The Material Stiffness

In matrix form, the stress train relationship is

$$
   \begin{Bmatrix}
     {\sigma}_{11} \\
     {\sigma}_{22} \\
     {\sigma}_{33} \\
     {\sigma}_{12} \\
     {\sigma}_{23} \\
     {\sigma}_{13} \\
     {\sigma}_{21} \\
     {\sigma}_{32} \\
     {\sigma}_{31}
   \end{Bmatrix} =
   \begin{bmatrix}
     C_{1111} & C_{1122} & C_{1133} & C_{1112} & C_{1123} & C_{1113} & C_{1121} & C_{1132} & C_{1131} \\
     C_{2211} & C_{2222} & C_{2233} & C_{2212} & C_{2223} & C_{2213} & C_{2221} & C_{2232} & C_{2231} \\
     C_{3311} & C_{3322} & C_{3333} & C_{3312} & C_{3323} & C_{3313} & C_{3321} & C_{3332} & C_{3331} \\
     C_{1211} & C_{1222} & C_{1233} & C_{1212} & C_{1223} & C_{1213} & C_{1221} & C_{1232} & C_{1231} \\
     C_{2311} & C_{2322} & C_{2333} & C_{2312} & C_{2323} & C_{2313} & C_{2321} & C_{2332} & C_{2331} \\
     C_{1311} & C_{1322} & C_{1333} & C_{1312} & C_{1323} & C_{1313} & C_{1321} & C_{1332} & C_{1331} \\
     C_{2111} & C_{2122} & C_{2133} & C_{2212} & C_{2123} & C_{2213} & C_{2121} & C_{2132} & C_{2131} \\
     C_{3211} & C_{3222} & C_{3233} & C_{3212} & C_{3223} & C_{3213} & C_{3221} & C_{3232} & C_{3231} \\
     C_{3111} & C_{3122} & C_{3133} & C_{3312} & C_{3123} & C_{3113} & C_{3121} & C_{3132} & C_{3131}
   \end{bmatrix}
   \begin{Bmatrix}
     {\epsilon}_{11} \\
     {\epsilon}_{22} \\
     {\epsilon}_{33} \\
     {\epsilon}_{12} \\
     {\epsilon}_{23} \\
     {\epsilon}_{13} \\
     {\epsilon}_{21} \\
     {\epsilon}_{32} \\
     {\epsilon}_{31}
   \end{Bmatrix}
$$

Due to the symmetries of the stiffness and strain tensors ($C_{ijkl}=C_{ijlk}$, ${\epsilon}_{ij}={\epsilon}_{ji}$), the expression above can be simplified by removing the last three columns of $C_{ijkl}$:

$$
   \begin{Bmatrix}
     {\sigma}_{11} \\
     {\sigma}_{22} \\
     {\sigma}_{33} \\
     {\sigma}_{12} \\
     {\sigma}_{23} \\
     {\sigma}_{13} \\
     {\sigma}_{21} \\
     {\sigma}_{32} \\
     {\sigma}_{31}
   \end{Bmatrix} =
   \begin{bmatrix}
     C_{1111} & C_{1122} & C_{1133} & C_{1112} & C_{1123} & C_{1113} \\
     C_{2211} & C_{2222} & C_{2233} & C_{2212} & C_{2223} & C_{2213} \\
     C_{3311} & C_{3322} & C_{3333} & C_{3312} & C_{3323} & C_{3313} \\
     C_{1211} & C_{1222} & C_{1233} & C_{1212} & C_{1223} & C_{1213} \\
     C_{2311} & C_{2322} & C_{2333} & C_{2312} & C_{2323} & C_{2313} \\
     C_{1311} & C_{1322} & C_{1333} & C_{1312} & C_{1323} & C_{1313} \\
     C_{2111} & C_{2122} & C_{2133} & C_{2212} & C_{2123} & C_{2213} \\
     C_{3211} & C_{3222} & C_{3233} & C_{3212} & C_{3223} & C_{3213} \\
     C_{3111} & C_{3122} & C_{3133} & C_{3112} & C_{3123} & C_{3113}
   \end{bmatrix}
   \begin{Bmatrix}
     {\epsilon}_{11} \\
     {\epsilon}_{22} \\
     {\epsilon}_{33} \\
     2{\epsilon}_{12} \\
     2{\epsilon}_{23} \\
     2{\epsilon}_{13}
   \end{Bmatrix}
$$

Considering the symmetry of the stress tensor
(${\sigma}_{ij}={\sigma}_{ji}$) and the major symmetry of
$C_{ijkl}$ ($C_{ijkl}=C_{klij}$), the final three rows of
$C_{ijkl}$ may also be ommitted, resulting in the symmetric form

$$
   \begin{Bmatrix}
     {\sigma}_{11} \\
     {\sigma}_{22} \\
     {\sigma}_{33} \\
     {\sigma}_{12} \\
     {\sigma}_{23} \\
     {\sigma}_{13}
   \end{Bmatrix} =
   \begin{bmatrix}
     C_{1111} & C_{1122} & C_{1133} & C_{1112} & C_{1123} & C_{1113} \\
              & C_{2222} & C_{2233} & C_{2212} & C_{2223} & C_{2213} \\
              &          & C_{3333} & C_{3312} & C_{3323} & C_{3313} \\
              &          &          & C_{1212} & C_{1223} & C_{1213} \\
              &          &          &          & C_{2323} & C_{2313} \\
    \text{symm}&         &          &          &          & C_{1313} \\
   \end{bmatrix}
   \begin{Bmatrix}
     {\epsilon}_{11} \\
     {\epsilon}_{22} \\
     {\epsilon}_{33} \\
     2{\epsilon}_{12} \\
     2{\epsilon}_{23} \\
     2{\epsilon}_{13}
   \end{Bmatrix}
$$

Letting $\{{\sigma}_{1},{\sigma}_{2},{\sigma}_{3},
{\sigma}_{4}, {\sigma}_{5}, {\sigma}_{6}\}=
\{{\sigma}_{11},{\sigma}_{22},{\sigma}_{33},
{\sigma}_{12},{\sigma}_{23},{\sigma}_{13}\}$ and
$\{{\epsilon}_{1},{\epsilon}_{2},{\epsilon}_{3},
{\gamma}_4, {\gamma}_5, {\gamma}_6\}=
\{{\epsilon}_{11},{\epsilon}_{22},{\epsilon}_{33},2{\epsilon}_{12},2{\epsilon}_{23},2{\epsilon}_{13}\}$,
the above stress-strain relationship is

$$
   \begin{Bmatrix}
     {\sigma}_{1} \\
     {\sigma}_{2} \\
     {\sigma}_{3} \\
     {\sigma}_{4} \\
     {\sigma}_{5} \\
     {\sigma}_{6}
   \end{Bmatrix} =
   \begin{bmatrix}
     C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16} \\
            & C_{22} & C_{23} & C_{24} & C_{25} & C_{26} \\
            &        & C_{33} & C_{34} & C_{35} & C_{36} \\
            &        &        & C_{44} & C_{45} & C_{46} \\
            &        &        &        & C_{55} & C_{56} \\
    \text{symm}   &        &        &        &        & C_{66} \\
   \end{bmatrix}
   \begin{Bmatrix}
     {\epsilon}_{1} \\
     {\epsilon}_{2} \\
     {\epsilon}_{3} \\
     {\gamma}_4 \\
     {\gamma}_5 \\
     {\gamma}_6
   \end{Bmatrix}
$$

The strain components $\gamma_i$ are the so called engineering shear strain components.
As expressed, the components of ${\epsilon}_{J}$ and ${\sigma}_{I}$ are first order tensors and $C_{IJ}$ is a second order tensor in $\mathbb{R}^{6}$, respectively.

<div style="background-color: #FFF8C6; margin-left: 20px; margin-right: 20px; padding-bottom: 8px; padding-left: 8px; padding-right: 8px; padding-top: 8px;"><center>Matmodlab passes engineering shear strains to material models</center>
</div>

<a name='implement'></a>
## Matmodlab Implementation

Below, a elastic material model is implemented as a standard Matmodlab material, subclassing the base `MaterialModel` class.  The model defines the following (required) attributes and methods

- `name`: name by which the model is referenced   
- `param_names`: method returning names of model parameters
- `update_state`: method the updates the material state
- `setup`: checks user input

The relevant input parameters from Matmodlab are:

- `stran`: the strain at the end of the time step.  The definition of the strain depends on the user input `kappa` for each step
   
### Loading the Material Model

Once defined in a computational cell, the material model is loaded in to Matmodlab through the `load_material` function using the `std_material` keyword.

In [1]:
%matmodlab

BokehJS successfully loaded.

Populating the interactive namespace from matmodlab and bokeh


In [2]:
import logging
from numpy import dot, zeros, ix_, eye
from numpy.linalg import cholesky, LinAlgError
from matmodlab.mmd.material import MaterialModel

class UserElastic(MaterialModel):
    name = "uelastic-std"

    @classmethod
    def param_names(cls, n):
        return ['C{0}{1}'.format(i+1,j+1) 
                for i in range(6) for j in range(i,6)]

    def setup(self, **kwargs):
        """Set up the Elastic material

        """
        logger = logging.getLogger('matmodlab.mmd.simulator')
        C = self.form_stiff(self.parameters)
        
        # check if C is positive definite
        try:
            np.linalg.cholesky(C)
        except LinAlgError:
            raise ValueError('elastic stiffness not positive definite')
    
    @staticmethod
    def form_stiff(cij):
        C = zeros((6,6))
        i = 0
        for k in range(6):
            j = 6 - k
            C[k, k:] = cij[i:i+j]
            i += j
        return C + np.triu(C,k=1).T
    
    def update_state(self, time, dtime, temp, dtemp, energy, rho, F0, F,
        stran, d, elec_field, stress, statev, **kwargs):
        """Compute updated stress given strain increment"""
        
        # elastic stiffness
        ddsdde = self.form_stiff(self.parameters)

        # stress update
        stress = dot(ddsdde, stran)

        return stress, statev, ddsdde
load_material(std_material=UserElastic)

<a name='verify'></a>
## Verification

Exercising the elastic model through a path of uniaxial stress using parameters corresponding to an isotropic stiffness tensor should result in the slope of axial stress vs. axial strain being equal to the input parameter `E`.

**Note:** the input parameters to a standard material are given as a dictionary of `name:value` pairs for each paramter.  Parameters not specified are initialized to a value of zero.

In [3]:
# Standard elastic moduli

E, Nu = 10e6, .333
K = E / 3. / (1. - 2. * Nu)
G = E / 2. / (1. + Nu)

K3 = 3. * K
G2 = 2. * G
Lam = (K3 - G2) / 3.

p = {}
p['C11'] = p['C22'] = p['C33'] = Lam + G2
p['C12'] = p['C13'] = p['C23'] = Lam
p['C44'] = p['C55'] = p['C66'] = G

mps1 = MaterialPointSimulator('uelastic-std')
mps1.Material('uelastic-std', p)
mps1.MixedStep(components=(.1, 0, 0), descriptors='ESS', frames=50)

i = where(mps1.E.XX > 0.)
E = mps1.S.XX[i] / mps1.E.XX[i]
assert allclose(E, 10e6, atol=1e-3, rtol=1e-3)